In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Define global constants for BC columns and wavelengths
BC_COLUMNS = [
    'UV BC1', 'UV BC2', 'UV BCc',
    'Blue BC1', 'Blue BC2', 'Blue BCc',
    'Green BC1', 'Green BC2', 'Green BCc',
    'Red BC1', 'Red BC2', 'Red BCc',
    'IR BC1', 'IR BC2', 'IR BCc'
]
WAVELENGTHS = {'UV': 375, 'Blue': 470, 'Green': 528, 'Red': 625, 'IR': 880}

In [2]:
def analyze_missing_data(filepath):
    # Part 1: Load the raw data
    print("Loading raw data...")
    raw_df = pd.read_csv(filepath)
    
    # Convert 'Time (UTC)' to datetime and then create local time
    raw_df['Time (UTC)'] = pd.to_datetime(raw_df['Time (UTC)'], utc=True)
    raw_df['Time (Local)'] = raw_df['Time (UTC)'].dt.tz_convert('Africa/Addis_Ababa')
    
    # Check for non-numeric values in BC columns and force conversion
    print("\nChecking for non-numeric values in data columns...")
    non_numeric_data = {}
    for col in BC_COLUMNS:
        if col in raw_df.columns:
            converted = pd.to_numeric(raw_df[col], errors='coerce')
            non_numeric_mask = converted.isna() & raw_df[col].notna()
            non_numeric_count = non_numeric_mask.sum()
            
            if non_numeric_count > 0:
                print(f"WARNING: Found {non_numeric_count} non-numeric values in {col}")
                non_numeric_data[col] = raw_df.loc[non_numeric_mask, col].value_counts().to_dict()
                print(f"Example non-numeric values: {list(non_numeric_data[col].keys())[:3]}")
                
            # Replace the column with its numeric conversion
            raw_df[col] = converted
    
    # Part 2: Data processing steps
    print("\nProcessing data...")
    processed_df = raw_df.copy()
    
    removed_data = {
        'non_numeric_values': non_numeric_data,
        'negative_values': {},
        'outliers': {}
    }
    
    # Convert from ng/m³ to µg/m³ for BC columns
    for col in BC_COLUMNS:
        if col in processed_df.columns:
            processed_df[col] = processed_df[col] / 1000
        else:
            print(f"Warning: Column {col} not found - skipping")
    
    # Remove negative values
    for col in BC_COLUMNS:
        if col in processed_df.columns:
            neg_mask = processed_df[col] < 0
            removed_data['negative_values'][col] = neg_mask.sum()
            processed_df.loc[neg_mask, col] = np.nan
    
    # Remove outliers (values greater than mean + 3*std)
    for col in BC_COLUMNS:
        if col in processed_df.columns:
            mean = processed_df[col].mean()
            std = processed_df[col].std()
            upper_limit = mean + 3 * std
            outlier_mask = processed_df[col] > upper_limit
            removed_data['outliers'][col] = outlier_mask.sum()
            processed_df.loc[outlier_mask, col] = np.nan
            
    print("\nNegative values removed:")
    for col, count in removed_data['negative_values'].items():
        print(f"{col}: {count} values")
    
    print("\nOutliers removed:")
    for col, count in removed_data['outliers'].items():
        print(f"{col}: {count} values")
    
    # Part 3: Resample data to hourly averages
    processed_df.set_index('Time (Local)', inplace=True)
    hourly_df = processed_df.resample('H').mean(numeric_only=True)
    
    # Analyze missing hours in the resampled data
    print("\nAnalyzing missing hours in the resampled data...")
    start_date = hourly_df.index.min()
    end_date = hourly_df.index.max()
    full_date_range = pd.date_range(start=start_date, end=end_date, freq='H')
    full_df = pd.DataFrame(index=full_date_range)
    merged_df = full_df.join(hourly_df)
    
    missing_hours = merged_df[BC_COLUMNS].isnull().sum()
    print(f"Missing hourly values (out of {len(full_date_range)} total hours):")
    print(missing_hours)
    
    completely_missing = merged_df[BC_COLUMNS].isnull().all(axis=1)
    print(f"\nCompletely missing hours (all columns): {completely_missing.sum()}")
    
    # Identify consecutive missing periods
    missing_periods = []
    current_period = None
    for timestamp, is_missing in completely_missing.items():
        if is_missing:
            if current_period is None:
                current_period = {"start": timestamp, "end": timestamp, "count": 1}
            else:
                current_period["end"] = timestamp
                current_period["count"] += 1
        else:
            if current_period is not None:
                missing_periods.append(current_period)
                current_period = None
    if current_period is not None:
        missing_periods.append(current_period)
    
    missing_periods.sort(key=lambda x: x["count"], reverse=True)
    print("\nLongest periods of completely missing data:")
    for i, period in enumerate(missing_periods[:10]):
        print(f"{i+1}. From {period['start']} to {period['end']} ({period['count']} hours)")
    
    return raw_df, processed_df, hourly_df, merged_df, missing_periods, removed_data

def visualize_missing_data(merged_df, bc_columns, missing_periods, removed_data):
    # Plot 1: Daily missing data heatmap (separate figure)
    plt.figure(figsize=(12, 6))
    missing_data = merged_df[bc_columns].isnull()
    daily_missing = missing_data.resample('D').mean()
    sns.heatmap(daily_missing.T, cmap='viridis', cbar_kws={'label': 'Fraction missing'})
    plt.title('Missing Data Heatmap (Daily)')
    plt.ylabel('BC Columns')
    plt.xlabel('Date')
    plt.tight_layout()
    plt.savefig('missing_data_heatmap_daily.png')
    plt.close()
    
    # Plot 2: Bar chart of missing values by column
    plt.figure(figsize=(12, 6))
    missing_counts = merged_df[bc_columns].isnull().sum().sort_values(ascending=False)
    missing_counts.plot(kind='bar')
    plt.title('Missing Values by Column')
    plt.ylabel('Missing Count')
    plt.xlabel('BC Column')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('missing_values_by_column.png')
    plt.close()
    
    # Plot 3: Missing data by hour of day
    plt.figure(figsize=(12, 6))
    hourly_missing = merged_df[bc_columns].isnull().groupby(merged_df.index.hour).mean()
    sns.heatmap(hourly_missing.T, cmap='viridis', cbar_kws={'label': 'Fraction missing'})
    plt.title('Missing Data by Hour of Day')
    plt.ylabel('BC Columns')
    plt.xlabel('Hour of Day')
    plt.tight_layout()
    plt.savefig('missing_data_by_hour.png')
    plt.close()
    
    print("Missing data visualizations saved as separate files.")

def alternative_resampling_methods(raw_df, bc_columns):
    print("\nExploring alternative resampling methods...")
    raw_df_copy = raw_df.copy()
    raw_df_copy.set_index('Time (Local)', inplace=True)
    
    # Ensure BC columns are numeric
    for col in bc_columns:
        if col in raw_df_copy.columns:
            raw_df_copy[col] = pd.to_numeric(raw_df_copy[col], errors='coerce')
    
    daily_df = raw_df_copy[bc_columns].resample('D').mean()
    daily_completeness = 1 - daily_df.isnull().mean()
    
    two_hour_df = raw_df_copy[bc_columns].resample('2H').mean()
    two_hour_completeness = 1 - two_hour_df.isnull().mean()
    
    # FIXED: use bc_columns (not bc_COLUMNS) to avoid NameError
    thirty_min_df = raw_df_copy[bc_columns].resample('30T').mean()
    thirty_min_completeness = 1 - thirty_min_df.isnull().mean()
    
    methods_comparison = pd.DataFrame({
        'Daily': daily_completeness,
        '2-Hour': two_hour_completeness,
        '30-Minute': thirty_min_completeness
    })
    
    print("\nData completeness comparison:")
    print(methods_comparison)
    
    # Plot the completeness comparison in a separate figure
    plt.figure(figsize=(12, 6))
    methods_comparison.plot(kind='bar')
    plt.title('Data Completeness by Resampling Method')
    plt.ylabel('Completeness (1.0 = complete)')
    plt.xlabel('BC Column')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('resampling_methods_comparison.png')
    plt.close()
    
    return daily_df, two_hour_df, thirty_min_df, methods_comparison

def calculate_diurnal_variations(raw_df, bc_columns, wavelengths):
    print("\nCalculating diurnal variations...")
    df_copy = raw_df.copy()
    df_copy.set_index('Time (Local)', inplace=True)
    
    for col in bc_columns:
        if col in df_copy.columns:
            df_copy[col] = pd.to_numeric(df_copy[col], errors='coerce')
    
    hourly_df = df_copy[bc_columns].resample('H').mean(numeric_only=True)
    hourly_df['Hour'] = hourly_df.index.hour
    diurnal_pattern = hourly_df.groupby('Hour')[bc_columns].mean()
    
    df_copy['Hour'] = df_copy.index.hour
    minute_diurnal = df_copy.groupby('Hour')[bc_columns].mean()
    
    # Plot 1: Diurnal variation from minute-level data (separate figure)
    plt.figure(figsize=(12, 6))
    selected_columns = ['UV BCc', 'Blue BCc', 'Green BCc', 'Red BCc', 'IR BCc']
    for col in selected_columns:
        if col in minute_diurnal.columns:
            plt.plot(minute_diurnal.index, minute_diurnal[col], label=f"{col.split()[0]} ({wavelengths[col.split()[0]]} nm)")
    plt.title('Diurnal Variation (Minute-Level Data)')
    plt.xlabel('Hour of Day')
    plt.ylabel('BC (µg/m³)')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.xticks(range(0, 24))
    plt.tight_layout()
    plt.savefig('diurnal_variation_minute.png')
    plt.close()
    
    # Plot 2: Diurnal variation from hourly data (separate figure)
    plt.figure(figsize=(12, 6))
    for col in selected_columns:
        if col in diurnal_pattern.columns:
            plt.plot(diurnal_pattern.index, diurnal_pattern[col], label=f"{col.split()[0]} ({wavelengths[col.split()[0]]} nm)")
    plt.title('Diurnal Variation (Hourly Data)')
    plt.xlabel('Hour of Day')
    plt.ylabel('BC (µg/m³)')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.xticks(range(0, 24))
    plt.tight_layout()
    plt.savefig('diurnal_variation_hourly.png')
    plt.close()
    
    return diurnal_pattern, minute_diurnal

def extract_original_data_issues(filepath):
    print("\nExtracting problematic data examples...")
    raw_df = pd.read_csv(filepath)
    
    problematic_rows = {}
    for col in BC_COLUMNS:
        if col in raw_df.columns:
            temp_series = pd.to_numeric(raw_df[col], errors='coerce')
            non_numeric_mask = temp_series.isna() & raw_df[col].notna()
            if non_numeric_mask.sum() > 0:
                key = f"{col} (non-numeric)"
                problematic_rows[key] = raw_df.loc[non_numeric_mask].head(5)
                sample_file = f"problematic_data_{col.replace(' ', '_')}.csv"
                problematic_rows[key].to_csv(sample_file, index=False)
                print(f"Saved examples of non-numeric data from {col} to {sample_file}")
    return problematic_rows

def full_analysis(filepath):
    # Extract examples of problematic data
    problematic_rows = extract_original_data_issues(filepath)
    
    # Analyze missing data and processing steps
    raw_df, processed_df, hourly_df, merged_df, missing_periods, removed_data = analyze_missing_data(filepath)
    
    # Visualize missing data (graphs saved separately)
    visualize_missing_data(merged_df, BC_COLUMNS, missing_periods, removed_data)
    
    # Explore alternative resampling methods
    daily_df, two_hour_df, thirty_min_df, methods_comparison = alternative_resampling_methods(raw_df, BC_COLUMNS)
    
    # Calculate diurnal variations
    diurnal_pattern, minute_diurnal = calculate_diurnal_variations(raw_df, BC_COLUMNS, WAVELENGTHS)
    
    # Create a simple report
    hourly_completeness = merged_df[BC_COLUMNS].count() / len(merged_df)
    daily_completeness_actual = daily_df[BC_COLUMNS].count() / len(daily_df)
    
    with open("missing_data_analysis_report.txt", "w") as f:
        f.write("BC Data Analysis Report\n")
        f.write("======================\n\n")
        
        f.write("1. DATA CLEANING SUMMARY\n")
        f.write("----------------------\n")
        f.write(f"- Non-numeric values found: {sum(len(v) for v in removed_data['non_numeric_values'].values()) if removed_data['non_numeric_values'] else 0}\n")
        f.write(f"- Negative values removed: {sum(removed_data['negative_values'].values())}\n")
        f.write(f"- Outliers removed: {sum(removed_data['outliers'].values())}\n\n")
        
        f.write("2. MISSING DATA ANALYSIS\n")
        f.write("----------------------\n")
        f.write(f"- Total timespan: {merged_df.index.min()} to {merged_df.index.max()}\n")
        f.write(f"- Total hours in timespan: {len(merged_df)}\n")
        completely_missing = merged_df[BC_COLUMNS].isnull().all(axis=1).sum()
        f.write(f"- Completely missing hours (all columns): {completely_missing} ({completely_missing/len(merged_df)*100:.1f}%)\n")
        
        f.write("\n3. LONGEST MISSING PERIODS\n")
        f.write("------------------------\n")
        for i, period in enumerate(missing_periods[:5]):
            f.write(f"- Period {i+1}: {period['start']} to {period['end']} ({period['count']} hours)\n")
        
        f.write("\n4. DATA COMPLETENESS BY METHOD\n")
        f.write("----------------------------\n")
        f.write("Hourly averages (original method):\n")
        for col, pct in hourly_completeness.items():
            f.write(f"  - {col}: {pct*100:.1f}%\n")
        
        f.write("\nDaily averages (direct method):\n")
        for col, pct in daily_completeness_actual.items():
            f.write(f"  - {col}: {pct*100:.1f}%\n")
        
        f.write("\n5. RECOMMENDATIONS\n")
        f.write("-----------------------------------\n")
        hourly_avg_completeness = hourly_completeness.mean()
        daily_avg_completeness = daily_completeness_actual.mean()
        
        if daily_avg_completeness > hourly_avg_completeness:
            f.write("1. Use direct aggregation from minute-level data to daily averages as suggested by Prof. Ann.\n")
            f.write(f"   - Data completeness improved from {hourly_avg_completeness*100:.1f}% to {daily_avg_completeness*100:.1f}%\n")
        else:
            f.write("1. Hourly averaging gives similar completeness to daily averaging.\n")
        
        f.write("2. For diurnal analysis, consider grouping minute-level data by hour directly to retain more data points.\n")
        f.write("3. Note periods with completely missing data as potential limitations in the analysis.\n")
    
    print("Full analysis complete. Check generated visualizations and 'missing_data_analysis_report.txt'.")
    
    return {
        'raw_df': raw_df,
        'processed_df': processed_df,
        'hourly_df': hourly_df,
        'merged_df': merged_df,
        'missing_periods': missing_periods,
        'removed_data': removed_data,
        'daily_df': daily_df,
        'two_hour_df': two_hour_df,
        'thirty_min_df': thirty_min_df,
        'methods_comparison': methods_comparison,
        'diurnal_pattern': diurnal_pattern,
        'minute_diurnal': minute_diurnal,
        'problematic_rows': problematic_rows
    }

if __name__ == "__main__":
    filepath = "/Users/ahmadjalil/Library/CloudStorage/GoogleDrive-ahzs645@gmail.com/My Drive/University/Research/Grad/UC Davis Ann/NASA MAIA/Data/Aethelometry Data/Jacros_MA350_1-min_2022-2024_Cleaned.csv"
    results = full_analysis(filepath)



Extracting problematic data examples...
Loading raw data...

Checking for non-numeric values in data columns...

Processing data...

Negative values removed:
UV BC1: 1116 values
UV BC2: 2034 values
UV BCc: 26701 values
Blue BC1: 791 values
Blue BC2: 657 values
Blue BCc: 18394 values
Green BC1: 1318 values
Green BC2: 1072 values
Green BCc: 19824 values
Red BC1: 919 values
Red BC2: 721 values
Red BCc: 21969 values
IR BC1: 1104 values
IR BC2: 597 values
IR BCc: 30983 values

Outliers removed:
UV BC1: 22312 values
UV BC2: 22918 values
UV BCc: 22942 values
Blue BC1: 22512 values
Blue BC2: 23093 values
Blue BCc: 140 values
Green BC1: 22646 values
Green BC2: 23171 values
Green BCc: 16717 values
Red BC1: 22804 values
Red BC2: 23329 values
Red BCc: 21484 values
IR BC1: 23150 values
IR BC2: 23615 values
IR BCc: 22771 values


/var/folders/q8/4q55gl35679357bs417130z40000gn/T/ipykernel_57980/4167265038.py:71: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly_df = processed_df.resample('H').mean(numeric_only=True)



Analyzing missing hours in the resampled data...
Missing hourly values (out of 20665 total hours):
UV BC1       2367
UV BC2       2367
UV BCc       2366
Blue BC1     2363
Blue BC2     2362
Blue BCc     2340
Green BC1    2363
Green BC2    2366
Green BCc    2352
Red BC1      2365
Red BC2      2368
Red BCc      2352
IR BC1       2367
IR BC2       2368
IR BCc       2345
dtype: int64

Completely missing hours (all columns): 2339

Longest periods of completely missing data:
1. From 2022-06-23 23:00:00+03:00 to 2022-07-27 10:00:00+03:00 (804 hours)
2. From 2023-06-21 07:00:00+03:00 to 2023-06-29 09:00:00+03:00 (195 hours)
3. From 2023-12-06 08:00:00+03:00 to 2023-12-12 09:00:00+03:00 (146 hours)
4. From 2023-01-17 21:00:00+03:00 to 2023-01-23 04:00:00+03:00 (128 hours)
5. From 2024-04-04 01:00:00+03:00 to 2024-04-08 20:00:00+03:00 (116 hours)
6. From 2024-07-25 19:00:00+03:00 to 2024-07-30 10:00:00+03:00 (112 hours)
7. From 2023-02-18 08:00:00+03:00 to 2023-02-22 06:00:00+03:00 (95 hours)
8.

/var/folders/q8/4q55gl35679357bs417130z40000gn/T/ipykernel_57980/4167265038.py:77: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_date_range = pd.date_range(start=start_date, end=end_date, freq='H')


Missing data visualizations saved as separate files.

Exploring alternative resampling methods...


/var/folders/q8/4q55gl35679357bs417130z40000gn/T/ipykernel_57980/4167265038.py:163: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  two_hour_df = raw_df_copy[bc_columns].resample('2H').mean()
/var/folders/q8/4q55gl35679357bs417130z40000gn/T/ipykernel_57980/4167265038.py:167: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  thirty_min_df = raw_df_copy[bc_columns].resample('30T').mean()



Data completeness comparison:
              Daily    2-Hour  30-Minute
UV BC1     0.922274  0.888416   0.885864
UV BC2     0.922274  0.888416   0.885864
UV BCc     0.922274  0.888416   0.885864
Blue BC1   0.922274  0.888416   0.885864
Blue BC2   0.922274  0.888416   0.885864
Blue BCc   0.922274  0.888416   0.885864
Green BC1  0.922274  0.888416   0.885864
Green BC2  0.922274  0.888416   0.885864
Green BCc  0.922274  0.888416   0.885864
Red BC1    0.922274  0.888416   0.885864
Red BC2    0.922274  0.888416   0.885864
Red BCc    0.922274  0.888416   0.885864
IR BC1     0.922274  0.888416   0.885864
IR BC2     0.922274  0.888416   0.885864
IR BCc     0.922274  0.888416   0.885864

Calculating diurnal variations...


/var/folders/q8/4q55gl35679357bs417130z40000gn/T/ipykernel_57980/4167265038.py:201: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly_df = df_copy[bc_columns].resample('H').mean(numeric_only=True)


Full analysis complete. Check generated visualizations and 'missing_data_analysis_report.txt'.


<Figure size 1200x600 with 0 Axes>

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Define global constants for BC columns and wavelengths
BC_COLUMNS = [
    'UV BC1', 'UV BC2', 'UV BCc',
    'Blue BC1', 'Blue BC2', 'Blue BCc',
    'Green BC1', 'Green BC2', 'Green BCc',
    'Red BC1', 'Red BC2', 'Red BCc',
    'IR BC1', 'IR BC2', 'IR BCc'
]

# Add specific attenuation cross-section values from Cai et al. (2013)
WAVELENGTHS = {
    'UV': {'nm': 375, 'sigma_atn': 24.069}, 
    'Blue': {'nm': 470, 'sigma_atn': 19.070}, 
    'Green': {'nm': 528, 'sigma_atn': 17.028}, 
    'Red': {'nm': 625, 'sigma_atn': 14.091}, 
    'IR': {'nm': 880, 'sigma_atn': 10.120}
}

def apply_cai_algorithms(raw_df, flow_rate=50, time_base=5, threshold_a=None, threshold_b=75):
    """
    Apply Cai et al. (2013) algorithms to identify suspect data points
    
    Parameters:
    -----------
    raw_df : pandas DataFrame
        DataFrame containing raw data with 'Sen' and 'Ref' columns
    flow_rate : int
        Flow rate in ml/min (default: 50)
    time_base : int
        Time base in minutes (default: 5)
    threshold_a : int or None
        Threshold for Algorithm 1 (default: None, will be set based on flow_rate and time_base)
    threshold_b : int
        Threshold for Algorithm 2 (default: 75, as per Cai et al.)
        
    Returns:
    --------
    df_with_flags : pandas DataFrame
        Original DataFrame with added flag column
    """
    print("\nApplying Cai et al. (2013) algorithms to identify suspect data points...")
    
    # Create a copy of the DataFrame
    df = raw_df.copy()
    
    # Set default threshold_a based on flow_rate and time_base if not provided
    if threshold_a is None:
        if flow_rate == 50 and time_base == 5:
            threshold_a = 750
        elif flow_rate == 100 and time_base == 1:
            threshold_a = 175
        else:
            # For other settings, use the 50 ml/min and 5 min setting as default
            threshold_a = 750
            print(f"Warning: Using default threshold_a={threshold_a} for flow_rate={flow_rate} and time_base={time_base}")
    
    # Check if 'Sen' and 'Ref' columns exist
    if 'Sen' not in df.columns or 'Ref' not in df.columns:
        raise ValueError("DataFrame must contain 'Sen' and 'Ref' columns")
    
    # Calculate ΔRef and ΔSen
    df['ΔRef'] = df['Ref'].diff()
    df['ΔSen'] = df['Sen'].diff()
    
    # Apply Algorithm 1: |ΔRef| > Threshold A
    algo1_flag = abs(df['ΔRef']) > threshold_a
    
    # Apply Algorithm 2: (ΔSen - ΔRef) > Threshold B
    algo2_flag = (df['ΔSen'] - df['ΔRef']) > threshold_b
    
    # Combine flags
    df['flag'] = algo1_flag | algo2_flag
    
    # Flag the subsequent point as well (as per Cai et al.)
    df['flag'] = df['flag'] | df['flag'].shift(-1).fillna(False)
    
    # Count flagged points
    flagged_count = df['flag'].sum()
    total_count = len(df)
    flagged_percent = (flagged_count / total_count) * 100
    
    print(f"Flagged {flagged_count} out of {total_count} data points ({flagged_percent:.1f}%)")
    
    return df

def clean_data_with_cai_methods(df_with_flags, option=1, bc_col='BC'):
    """
    Apply data cleaning methods as per Cai et al. (2013)
    
    Parameters:
    -----------
    df_with_flags : pandas DataFrame
        DataFrame with flag column from apply_cai_algorithms
    option : int
        Cleaning option (1, 2, or 3) as per Cai et al. (2013)
    bc_col : str
        Column name for BC data
        
    Returns:
    --------
    cleaned_df : pandas DataFrame
        DataFrame with cleaned BC data
    """
    print(f"\nApplying data cleaning option {option}...")
    
    # Create a copy of the DataFrame
    df = df_with_flags.copy()
    
    # Create a new column for cleaned BC
    cleaned_col = f"{bc_col}_cleaned"
    df[cleaned_col] = df[bc_col].copy()
    
    # Get indices of flagged points
    flagged_indices = df[df['flag']].index
    
    if option == 1:
        # Option 1: Remove flagged data points
        df.loc[flagged_indices, cleaned_col] = np.nan
        print(f"Option 1: Set {len(flagged_indices)} flagged points to NaN")
        
    elif option == 2:
        # Option 2: Interpolate between points before and after
        for idx in flagged_indices:
            # Find the nearest non-flagged points before and after
            prev_idx = df.index[df.index < idx]
            if len(prev_idx) > 0:
                prev_idx = prev_idx[-1]
                while prev_idx in flagged_indices and prev_idx > df.index[0]:
                    temp_prev = df.index[df.index < prev_idx]
                    if len(temp_prev) > 0:
                        prev_idx = temp_prev[-1]
                    else:
                        break
            
            next_idx = df.index[df.index > idx]
            if len(next_idx) > 0:
                next_idx = next_idx[0]
                while next_idx in flagged_indices and next_idx < df.index[-1]:
                    temp_next = df.index[df.index > next_idx]
                    if len(temp_next) > 0:
                        next_idx = temp_next[0]
                    else:
                        break
            
            # If valid prev and next indices found, interpolate
            if len(prev_idx) > 0 and len(next_idx) > 0 and prev_idx not in flagged_indices and next_idx not in flagged_indices:
                df.loc[idx, cleaned_col] = (df.loc[prev_idx, bc_col] + df.loc[next_idx, bc_col]) / 2
            else:
                df.loc[idx, cleaned_col] = np.nan
        
        print(f"Option 2: Interpolated {len(flagged_indices)} flagged points")
        
    elif option == 3:
        # Option 3: Physical model using ATN change
        if 'ATN' not in df.columns:
            print("Warning: ATN column not found, calculating from Sen/Ref")
            df['ATN'] = -100 * np.log(df['Sen'] / df['Ref'])
        
        for idx in flagged_indices:
            # Find the nearest non-flagged points before and after
            prev_idx = df.index[df.index < idx]
            if len(prev_idx) > 0:
                prev_idx = prev_idx[-1]
                while prev_idx in flagged_indices and prev_idx > df.index[0]:
                    temp_prev = df.index[df.index < prev_idx]
                    if len(temp_prev) > 0:
                        prev_idx = temp_prev[-1]
                    else:
                        break
            
            next_idx = df.index[df.index > idx]
            if len(next_idx) > 0:
                next_idx = next_idx[0]
                while next_idx in flagged_indices and next_idx < df.index[-1]:
                    temp_next = df.index[df.index > next_idx]
                    if len(temp_next) > 0:
                        next_idx = temp_next[0]
                    else:
                        break
            
            # If valid prev and next indices found, use ATN change
            if len(prev_idx) > 0 and len(next_idx) > 0 and prev_idx not in flagged_indices and next_idx not in flagged_indices:
                delta_atn = df.loc[next_idx, 'ATN'] - df.loc[prev_idx, 'ATN']
                delta_t = (next_idx - prev_idx).total_seconds() / 60  # in minutes
                
                # Assuming WAVELENGTHS contains sigma_atn in m²/g for the wavelength
                # Default to IR wavelength if specific wavelength not identifiable
                sigma_atn = WAVELENGTHS.get('IR', {}).get('sigma_atn', 10.120)
                
                # Extract wavelength info from column name if possible
                for wave_key in WAVELENGTHS:
                    if wave_key in bc_col:
                        sigma_atn = WAVELENGTHS[wave_key]['sigma_atn']
                        break
                
                # Q is flow rate in m³/min
                Q = flow_rate / 1000000  # Convert ml/min to m³/min
                
                # Calculate BC using physical model
                if delta_t > 0:
                    bc = delta_atn / (sigma_atn * Q * delta_t)
                    df.loc[idx, cleaned_col] = bc
                else:
                    df.loc[idx, cleaned_col] = np.nan
            else:
                df.loc[idx, cleaned_col] = np.nan
        
        print(f"Option 3: Applied physical model to {len(flagged_indices)} flagged points")
    
    else:
        raise ValueError("Option must be 1, 2, or 3")
    
    return df

def analyze_missing_data_enhanced(filepath, flow_rate=50, time_base=5):
    """
    Enhanced version of your analyze_missing_data function that incorporates Cai et al. algorithms
    """
    # Part 1: Load the raw data (same as your original function)
    print("Loading raw data...")
    raw_df = pd.read_csv(filepath)
    
    # Convert 'Time (UTC)' to datetime and then create local time
    raw_df['Time (UTC)'] = pd.to_datetime(raw_df['Time (UTC)'], utc=True)
    raw_df['Time (Local)'] = raw_df['Time (UTC)'].dt.tz_convert('Africa/Addis_Ababa')
    
    # Check for non-numeric values in BC columns and force conversion
    print("\nChecking for non-numeric values in data columns...")
    non_numeric_data = {}
    for col in BC_COLUMNS:
        if col in raw_df.columns:
            converted = pd.to_numeric(raw_df[col], errors='coerce')
            non_numeric_mask = converted.isna() & raw_df[col].notna()
            non_numeric_count = non_numeric_mask.sum()
            
            if non_numeric_count > 0:
                print(f"WARNING: Found {non_numeric_count} non-numeric values in {col}")
                non_numeric_data[col] = raw_df.loc[non_numeric_mask, col].value_counts().to_dict()
                print(f"Example non-numeric values: {list(non_numeric_data[col].keys())[:3]}")
                
            # Replace the column with its numeric conversion
            raw_df[col] = converted
    
    # NEW: Apply Cai et al. algorithms if Sen and Ref columns exist
    if 'Sen' in raw_df.columns and 'Ref' in raw_df.columns:
        # Apply algorithms to each BC column
        flagged_dfs = {}
        for col in BC_COLUMNS:
            if col in raw_df.columns:
                # Create subset with just this BC column and Sen/Ref
                subset_df = raw_df[['Time (Local)', col, 'Sen', 'Ref']].copy()
                flagged_df = apply_cai_algorithms(subset_df, flow_rate=flow_rate, time_base=time_base)
                
                # Apply all three cleaning options for comparison
                for option in [1, 2, 3]:
                    cleaned_df = clean_data_with_cai_methods(flagged_df, option=option, bc_col=col)
                    raw_df[f"{col}_cleaned_opt{option}"] = cleaned_df[f"{col}_cleaned"]
                
                flagged_dfs[col] = flagged_df
        
        # Create visualization to compare raw vs cleaned data
        for col in BC_COLUMNS[:1]:  # Just do the first column for example
            if col in raw_df.columns:
                plt.figure(figsize=(12, 8))
                plt.plot(raw_df['Time (Local)'], raw_df[col], 'b-', alpha=0.5, label='Raw')
                plt.plot(raw_df['Time (Local)'], raw_df[f"{col}_cleaned_opt1"], 'r-', label='Cleaned (Option 1)')
                plt.plot(raw_df['Time (Local)'], raw_df[f"{col}_cleaned_opt2"], 'g-', label='Cleaned (Option 2)')
                plt.plot(raw_df['Time (Local)'], raw_df[f"{col}_cleaned_opt3"], 'y-', label='Cleaned (Option 3)')
                plt.title(f'Comparison of Raw vs Cleaned Data for {col}')
                plt.xlabel('Time')
                plt.ylabel('BC (µg/m³)')
                plt.legend()
                plt.tight_layout()
                plt.savefig(f'cleaned_data_comparison_{col.replace(" ", "_")}.png')
                plt.close()
    else:
        print("Warning: Sen and Ref columns not found, cannot apply Cai et al. algorithms")
    
    # Continue with the rest of your original function...
    # Part 2: Data processing steps
    print("\nProcessing data...")
    processed_df = raw_df.copy()
    
    removed_data = {
        'non_numeric_values': non_numeric_data,
        'negative_values': {},
        'outliers': {}
    }
    
    # Convert from ng/m³ to µg/m³ for BC columns
    for col in BC_COLUMNS:
        if col in processed_df.columns:
            processed_df[col] = processed_df[col] / 1000
        else:
            print(f"Warning: Column {col} not found - skipping")
    
    # Remove negative values
    for col in BC_COLUMNS:
        if col in processed_df.columns:
            neg_mask = processed_df[col] < 0
            removed_data['negative_values'][col] = neg_mask.sum()
            processed_df.loc[neg_mask, col] = np.nan
    
    # Remove outliers (values greater than mean + 3*std)
    for col in BC_COLUMNS:
        if col in processed_df.columns:
            mean = processed_df[col].mean()
            std = processed_df[col].std()
            upper_limit = mean + 3 * std
            outlier_mask = processed_df[col] > upper_limit
            removed_data['outliers'][col] = outlier_mask.sum()
            processed_df.loc[outlier_mask, col] = np.nan
            
    print("\nNegative values removed:")
    for col, count in removed_data['negative_values'].items():
        print(f"{col}: {count} values")
    
    print("\nOutliers removed:")
    for col, count in removed_data['outliers'].items():
        print(f"{col}: {count} values")
    
    # Part 3: Resample data to hourly averages
    processed_df.set_index('Time (Local)', inplace=True)
    hourly_df = processed_df.resample('H').mean(numeric_only=True)
    
    # Analyze missing hours in the resampled data
    print("\nAnalyzing missing hours in the resampled data...")
    start_date = hourly_df.index.min()
    end_date = hourly_df.index.max()
    full_date_range = pd.date_range(start=start_date, end=end_date, freq='H')
    full_df = pd.DataFrame(index=full_date_range)
    merged_df = full_df.join(hourly_df)
    
    # Include cleaned columns in the analysis
    all_bc_columns = BC_COLUMNS.copy()
    for col in BC_COLUMNS:
        for opt in [1, 2, 3]:
            cleaned_col = f"{col}_cleaned_opt{opt}"
            if cleaned_col in merged_df.columns:
                all_bc_columns.append(cleaned_col)
    
    missing_hours = merged_df[all_bc_columns].isnull().sum()
    print(f"Missing hourly values (out of {len(full_date_range)} total hours):")
    print(missing_hours)
    
    completely_missing = merged_df[BC_COLUMNS].isnull().all(axis=1)
    print(f"\nCompletely missing hours (all columns): {completely_missing.sum()}")
    
    # Identify consecutive missing periods
    missing_periods = []
    current_period = None
    for timestamp, is_missing in completely_missing.items():
        if is_missing:
            if current_period is None:
                current_period = {"start": timestamp, "end": timestamp, "count": 1}
            else:
                current_period["end"] = timestamp
                current_period["count"] += 1
        else:
            if current_period is not None:
                missing_periods.append(current_period)
                current_period = None
    if current_period is not None:
        missing_periods.append(current_period)
    
    missing_periods.sort(key=lambda x: x["count"], reverse=True)
    print("\nLongest periods of completely missing data:")
    for i, period in enumerate(missing_periods[:10]):
        print(f"{i+1}. From {period['start']} to {period['end']} ({period['count']} hours)")
    
    # Compare data completeness between raw and cleaned data
    if 'Sen' in raw_df.columns and 'Ref' in raw_df.columns:
        print("\nComparing data completeness: Raw vs. Cleaned")
        for col in BC_COLUMNS:
            if col in merged_df.columns:
                raw_complete = 100 - (merged_df[col].isnull().sum() / len(merged_df) * 100)
                for opt in [1, 2, 3]:
                    cleaned_col = f"{col}_cleaned_opt{opt}"
                    if cleaned_col in merged_df.columns:
                        cleaned_complete = 100 - (merged_df[cleaned_col].isnull().sum() / len(merged_df) * 100)
                        print(f"{col}: Raw={raw_complete:.1f}%, Cleaned (Opt {opt})={cleaned_complete:.1f}%")
    
    return raw_df, processed_df, hourly_df, merged_df, missing_periods, removed_data

def full_analysis_enhanced(filepath, flow_rate=50, time_base=5):
    """
    Enhanced version of your full_analysis function that incorporates Cai et al. algorithms
    """
    # Extract examples of problematic data
    problematic_rows = extract_original_data_issues(filepath)
    
    # Enhanced analyze_missing_data function
    raw_df, processed_df, hourly_df, merged_df, missing_periods, removed_data = analyze_missing_data_enhanced(filepath, flow_rate, time_base)
    
    # Continue with your original function components...
    # Visualize missing data
    visualize_missing_data(merged_df, BC_COLUMNS, missing_periods, removed_data)
    
    # Explore alternative resampling methods
    daily_df, two_hour_df, thirty_min_df, methods_comparison = alternative_resampling_methods(raw_df, BC_COLUMNS)
    
    # Calculate diurnal variations
    wavelengths_nm = {k: v['nm'] for k, v in WAVELENGTHS.items()}
    diurnal_pattern, minute_diurnal = calculate_diurnal_variations(raw_df, BC_COLUMNS, wavelengths_nm)
    
    # Create a simple report
    hourly_completeness = merged_df[BC_COLUMNS].count() / len(merged_df)
    daily_completeness_actual = daily_df[BC_COLUMNS].count() / len(daily_df)
    
    # Generate report including Cai et al. algorithm results
    with open("enhanced_bc_analysis_report.txt", "w") as f:
        f.write("Enhanced BC Data Analysis Report with Cai et al. Algorithms\n")
        f.write("====================================================\n\n")
        
        f.write("1. DATA CLEANING SUMMARY\n")
        f.write("----------------------\n")
        f.write(f"- Non-numeric values found: {sum(len(v) for v in removed_data['non_numeric_values'].values()) if removed_data['non_numeric_values'] else 0}\n")
        f.write(f"- Negative values removed: {sum(removed_data['negative_values'].values())}\n")
        f.write(f"- Outliers removed: {sum(removed_data['outliers'].values())}\n\n")
        
        f.write("2. CAI ET AL. ALGORITHM RESULTS\n")
        f.write("-----------------------------\n")
        if 'Sen' in raw_df.columns and 'Ref' in raw_df.columns:
            f.write("Applied Cai et al. (2013) algorithms for identifying suspect data points:\n")
            f.write("- Algorithm 1: |ΔRef| > Threshold A (identifies abnormal reference signal fluctuations)\n")
            f.write("- Algorithm 2: (ΔSen - ΔRef) > Threshold B (identifies when sensing deviates from reference)\n")
            f.write("- Three data processing options were applied and compared\n\n")
        else:
            f.write("Could not apply Cai et al. algorithms - 'Sen' and 'Ref' columns not found in the data\n\n")
        
        f.write("3. MISSING DATA ANALYSIS\n")
        f.write("----------------------\n")
        f.write(f"- Total timespan: {merged_df.index.min()} to {merged_df.index.max()}\n")
        f.write(f"- Total hours in timespan: {len(merged_df)}\n")
        completely_missing = merged_df[BC_COLUMNS].isnull().all(axis=1).sum()
        f.write(f"- Completely missing hours (all columns): {completely_missing} ({completely_missing/len(merged_df)*100:.1f}%)\n")
        
        f.write("\n4. LONGEST MISSING PERIODS\n")
        f.write("------------------------\n")
        for i, period in enumerate(missing_periods[:5]):
            f.write(f"- Period {i+1}: {period['start']} to {period['end']} ({period['count']} hours)\n")
        
        f.write("\n5. DATA COMPLETENESS BY METHOD\n")
        f.write("----------------------------\n")
        f.write("Hourly averages (original method):\n")
        for col, pct in hourly_completeness.items():
            f.write(f"  - {col}: {pct*100:.1f}%\n")
        
        f.write("\nDaily averages (direct method):\n")
        for col, pct in daily_completeness_actual.items():
            f.write(f"  - {col}: {pct*100:.1f}%\n")
        
        f.write("\n6. RECOMMENDATIONS\n")
        f.write("-----------------------------------\n")
        hourly_avg_completeness = hourly_completeness.mean()
        daily_avg_completeness = daily_completeness_actual.mean()
        
        f.write("1. Implement Cai et al. (2013) algorithms for identifying suspect data points:\n")
        f.write("   - Algorithm 1: |ΔRef| > Threshold A (where A ≈ 750 for 50 ml/min flow rate and 5 min time base)\n")
        f.write("   - Algorithm 2: (ΔSen - ΔRef) > Threshold B (where B ≈ 75)\n")
        
        f.write("\n2. For data cleaning, use Option 1 (removing flagged data) or Option 2 (interpolation) as they showed better reproducibility.\n")
        
        if daily_avg_completeness > hourly_avg_completeness:
            f.write("\n3. Use direct aggregation from minute-level data to daily averages as suggested by Prof. Ann.\n")
            f.write(f"   - Data completeness improved from {hourly_avg_completeness*100:.1f}% to {daily_avg_completeness*100:.1f}%\n")
        else:
            f.write("\n3. Hourly averaging gives similar completeness to daily averaging.\n")
        
        f.write("\n4. For diurnal analysis, consider grouping minute-level data by hour directly to retain more data points.\n")
        f.write("\n5. Note periods with completely missing data as potential limitations in the analysis.\n")
        
        f.write("\n6. When collecting new data, consider using diffusion driers if instruments will be used in environments with rapid humidity changes.\n")
    
    print("Enhanced analysis complete. Check generated visualizations and 'enhanced_bc_analysis_report.txt'.")
    
    return {
        'raw_df': raw_df,
        'processed_df': processed_df,
        'hourly_df': hourly_df,
        'merged_df': merged_df,
        'missing_periods': missing_periods,
        'removed_data': removed_data,
        'daily_df': daily_df,
        'two_hour_df': two_hour_df,
        'thirty_min_df': thirty_min_df,
        'methods_comparison': methods_comparison,
        'diurnal_pattern': diurnal_pattern,
        'minute_diurnal': minute_diurnal,
        'problematic_rows': problematic_rows
    }

# Example usage
if __name__ == "__main__":
    filepath = "/Users/ahmadjalil/Library/CloudStorage/GoogleDrive-ahzs645@gmail.com/My Drive/University/Research/Grad/UC Davis Ann/NASA MAIA/Data/Aethelometry Data/Jacros_MA350_1-min_2022-2024_Cleaned.csv"
    results = full_analysis_enhanced(filepath, flow_rate=50, time_base=5)


Extracting problematic data examples...
Loading raw data...

Checking for non-numeric values in data columns...

Processing data...

Negative values removed:
UV BC1: 1116 values
UV BC2: 2034 values
UV BCc: 26701 values
Blue BC1: 791 values
Blue BC2: 657 values
Blue BCc: 18394 values
Green BC1: 1318 values
Green BC2: 1072 values
Green BCc: 19824 values
Red BC1: 919 values
Red BC2: 721 values
Red BCc: 21969 values
IR BC1: 1104 values
IR BC2: 597 values
IR BCc: 30983 values

Outliers removed:
UV BC1: 22312 values
UV BC2: 22918 values
UV BCc: 22942 values
Blue BC1: 22512 values
Blue BC2: 23093 values
Blue BCc: 140 values
Green BC1: 22646 values
Green BC2: 23171 values
Green BCc: 16717 values
Red BC1: 22804 values
Red BC2: 23329 values
Red BCc: 21484 values
IR BC1: 23150 values
IR BC2: 23615 values
IR BCc: 22771 values


/var/folders/q8/4q55gl35679357bs417130z40000gn/T/ipykernel_57980/1015572904.py:330: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly_df = processed_df.resample('H').mean(numeric_only=True)



Analyzing missing hours in the resampled data...
Missing hourly values (out of 20665 total hours):
UV BC1       2367
UV BC2       2367
UV BCc       2366
Blue BC1     2363
Blue BC2     2362
Blue BCc     2340
Green BC1    2363
Green BC2    2366
Green BCc    2352
Red BC1      2365
Red BC2      2368
Red BCc      2352
IR BC1       2367
IR BC2       2368
IR BCc       2345
dtype: int64

Completely missing hours (all columns): 2339

Longest periods of completely missing data:
1. From 2022-06-23 23:00:00+03:00 to 2022-07-27 10:00:00+03:00 (804 hours)
2. From 2023-06-21 07:00:00+03:00 to 2023-06-29 09:00:00+03:00 (195 hours)
3. From 2023-12-06 08:00:00+03:00 to 2023-12-12 09:00:00+03:00 (146 hours)
4. From 2023-01-17 21:00:00+03:00 to 2023-01-23 04:00:00+03:00 (128 hours)
5. From 2024-04-04 01:00:00+03:00 to 2024-04-08 20:00:00+03:00 (116 hours)
6. From 2024-07-25 19:00:00+03:00 to 2024-07-30 10:00:00+03:00 (112 hours)
7. From 2023-02-18 08:00:00+03:00 to 2023-02-22 06:00:00+03:00 (95 hours)
8.

/var/folders/q8/4q55gl35679357bs417130z40000gn/T/ipykernel_57980/1015572904.py:336: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_date_range = pd.date_range(start=start_date, end=end_date, freq='H')


Missing data visualizations saved as separate files.

Exploring alternative resampling methods...


/var/folders/q8/4q55gl35679357bs417130z40000gn/T/ipykernel_57980/4167265038.py:163: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  two_hour_df = raw_df_copy[bc_columns].resample('2H').mean()
/var/folders/q8/4q55gl35679357bs417130z40000gn/T/ipykernel_57980/4167265038.py:167: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  thirty_min_df = raw_df_copy[bc_columns].resample('30T').mean()



Data completeness comparison:
              Daily    2-Hour  30-Minute
UV BC1     0.922274  0.888416   0.885864
UV BC2     0.922274  0.888416   0.885864
UV BCc     0.922274  0.888416   0.885864
Blue BC1   0.922274  0.888416   0.885864
Blue BC2   0.922274  0.888416   0.885864
Blue BCc   0.922274  0.888416   0.885864
Green BC1  0.922274  0.888416   0.885864
Green BC2  0.922274  0.888416   0.885864
Green BCc  0.922274  0.888416   0.885864
Red BC1    0.922274  0.888416   0.885864
Red BC2    0.922274  0.888416   0.885864
Red BCc    0.922274  0.888416   0.885864
IR BC1     0.922274  0.888416   0.885864
IR BC2     0.922274  0.888416   0.885864
IR BCc     0.922274  0.888416   0.885864

Calculating diurnal variations...


/var/folders/q8/4q55gl35679357bs417130z40000gn/T/ipykernel_57980/4167265038.py:201: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly_df = df_copy[bc_columns].resample('H').mean(numeric_only=True)


Enhanced analysis complete. Check generated visualizations and 'enhanced_bc_analysis_report.txt'.


<Figure size 1200x600 with 0 Axes>